# Final project: Step 1
## Data acquisition and data wrangling

In [2]:
#importar librerias
import pandas as pd
pd.set_option('display.max_columns', None)

import re

from tqdm import tqdm

import requests
import time
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.keys import Keys


### 1. First source: a public database of cities in the world (.csv)
https://www.kaggle.com/dataset/f66386cd35268fd2ae9c7c03e6e4d93c9b1607265c1adef13f99a76e420be997/version/1

In [3]:
#extraer CSV con base de datos de ciudades
cities = pd.read_csv('../Data/worldcities.csv')

In [5]:
#imprimir una muestra de la tabla
cities.head()

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.6850,139.7514,Japan,JP,JPN,Tōkyō,primary,35676000.0,1392685764
1,New York,New York,40.6943,-73.9249,United States,US,USA,New York,NaN,19354922.0,1840034016
2,Mexico City,Mexico City,19.4424,-99.1310,Mexico,MX,MEX,Ciudad de México,primary,19028000.0,1484247881
3,Mumbai,Mumbai,19.0170,72.8570,India,IN,IND,Mahārāshtra,admin,18978000.0,1356226629
4,São Paulo,Sao Paulo,-23.5587,-46.6250,Brazil,BR,BRA,São Paulo,admin,18845000.0,1076532519


In [6]:
#eliminamos columnas que no nos van a servir
cities.drop(['admin_name', 'capital',"id"], axis=1, inplace=True)

In [7]:
#renombramos las columnas
cities.columns = ['Original name', 'City', 'Lat', 'Lon', 'Country', 'ISO2', 'ISO3', 'Population']

In [17]:
#hay varias ciudades que estan duplicadas, eliminamos los registros repetidos y solo dejamos el primer registro
cities.drop_duplicates(subset=['City', 'Country'], keep='first',inplace=True)

In [18]:
#imprimir una muestra de la tabla
cities.head()

,Original name,City,Lat,Lon,Country,ISO2,ISO3,Population
0,Tokyo,Tokyo,35.6850,139.7514,Japan,JP,JPN,35676000.0
1,New York,New York,40.6943,-73.9249,United States,US,USA,19354922.0
2,Mexico City,Mexico City,19.4424,-99.1310,Mexico,MX,MEX,19028000.0
3,Mumbai,Mumbai,19.0170,72.8570,India,IN,IND,18978000.0
4,São Paulo,Sao Paulo,-23.5587,-46.6250,Brazil,BR,BRA,18845000.0


### 2. Second source: a public database of prices by city (web scrapping)
https://www.numbeo.com/cost-of-living/prices_by_city.jsp?displayCurrency=USD&itemId=118&itemId=15&itemId=11&itemId=13&itemId=1

In [20]:
#hacemos el request de la segunda fuente de datos (vamos a  sacar los datos con web srapping)
url = 'https://www.numbeo.com/cost-of-living/prices_by_city.jsp?displayCurrency=USD&itemId=118&itemId=15&itemId=11&itemId=13&itemId=1'
resp = requests.get(url)
sopa = bs(resp.content, "html.parser")

In [21]:
#llamamos a la tabla de la página
table = sopa.find("table",{"id":"t2"})

In [22]:
#sacamos cada fila de la tabla
filas = table.findAll("tr")

In [23]:
#eliminamos la primera fila (son los titulos de columna)
filas.pop(0)

<tr>
<th><div style="font-size: 80%; vertical-align: middle;">Rank</div></th>
<th><div class="font_in_table_headers">City</div></th><th><div class="font_in_table_headers">Meal, Inexpensive Restaurant</div></th><th><div class="font_in_table_headers">Eggs <br/>(regular) <br/>(12)</div></th><th><div class="font_in_table_headers">Water <br/>(1.5 liter bottle)</div></th><th><div class="font_in_table_headers">Domestic Beer <br/>(0.5 liter bottle)</div></th><th><div class="font_in_table_headers">Banana <br/>(1kg)</div></th></tr>

In [29]:
#generamos el data frame con la información
numbeo = []

for i in filas:
    city = i.findAll("td")[1].text.split(", ")[0]
    city = re.sub("\(.*\)","",city).strip()
    country = i.findAll("td")[1].text.split(", ")[-1].strip()
    country = re.sub("\(.*\)","",country).strip()
    meal = i.findAll("td")[2].text.strip()
    eggs = i.findAll("td")[3].text.strip()
    water = i.findAll("td")[4].text.strip()
    beer = i.findAll("td")[5].text.strip()
    banana = i.findAll("td")[6].text.strip()
    
    row = {"City": city,"Country": country, "Meal (Inexpensive Restaurant) (USD)": meal,"Eggs (12) (USD)": eggs,"Water (1.5 liter bottle) (USD)": water,"Domestic Beer (USD)": beer,"Banana (1kg) (USD)": banana}
    numbeo.append(row)
    
numbeo_df = pd.DataFrame(numbeo)
numbeo_df.head()

,City,Country,Meal (Inexpensive Restaurant) (USD),Eggs (12) (USD),Water (1.5 liter bottle) (USD),Domestic Beer (USD),Banana (1kg) (USD)
0,Saint Petersburg,Russia,6.56,1.11,0.56,0.86,0.84
1,Samara,Russia,6.62,0.95,0.43,0.71,0.89
2,Algiers,Algeria,3.11,1.15,0.24,1.79,1.88
3,Saratov,Russia,5.29,0.99,0.36,0.82,0.80
4,Banja Luka,Bosnia And Herzegovina,4.85,1.82,0.61,0.64,1.31


### 3. Third source: a public list of daily budget by city for backpackers (web scrapping)
https://www.priceoftravel.com/world-cities-by-price-backpacker-index/

In [30]:
#hacemos el request de la tercera fuente de datos (vamos a  sacar los datos con web srapping)
url = 'https://www.priceoftravel.com/world-cities-by-price-backpacker-index/'
resp = requests.get(url)
sopa = bs(resp.content, "html.parser")

In [31]:
#llamamos a los elementos que nos van a servir del html
lst = sopa.find("div",{"class":"bpiidx_list"}).findAllNext("div",{"id":"bpi_row1"})

In [32]:
#generamos el dataframe con los datos extraidos
index = []

for i in lst:
    city = i.findAll("div",{"class":"bpidx"})[2].text.split(", ")[0]
    city = re.sub("\(.*\)","",city).strip()
    country = i.findAll("div",{"class":"bpidx"})[2].text.split(", ")[-1].strip()
    budget = i.findAll("div",{"class":"bpidx"})[1].text
    
    row = {"City": city,"Country": country, "Daily Budget": budget}
    index.append(row)
    
index_df = pd.DataFrame(index)
index_df.head()

,City,Country,Daily Budget
0,Hanoi,Viet Nam,$19.70
1,Saigon,Viet Nam,$20.54
2,Vientiane,Laos,$21.07
3,Hoi An,Viet Nam,$21.48
4,Pokhara,Nepal,$21.71


### 4. Fourth source: Information (a lot) about cities around the world (web scrapping with Selenium)
https://nomadlist.com/

In [70]:
#abrimos el navegador de Selenium
navegador = webdriver.Chrome()

In [71]:
#maximizamos el navegador
navegador.maximize_window()

In [72]:
#vamos al url
navegador.get('https://nomadlist.com/userApi.php?action=login_by_email&hash=91e7c90f5dcbfc27710077645e4f4841b8949523')

In [75]:
#abrimos el menu
menu = navegador.find_element_by_xpath('/html/body/div[3]/div/div[1]/span')
menu.click()

In [76]:
#buscamos el home
home = navegador.find_element_by_xpath('/html/body/div[5]/a[1]')
home.click()

In [286]:
#lista de ciudades
#ciudades = []
#lst_ciudades = []

In [97]:
#Si hay dos archivos de checkpoint CSV (juntamos los archivos generados)
ciudades_df = pd.read_csv('nomadlist_cities.csv').drop('Unnamed: 0', axis=1)
ciudades2_df = pd.read_csv('nomadlist_cities2.csv').drop('Unnamed: 0', axis=1)

#generamos una lista de las ciudades para ir haciendo el append
ciudades = ciudades_df.to_dict('records') + ciudades2_df.to_dict('records')

#total de ciudades ya escrapeadas
print(len(ciudades))

#generamos una lista para verificar si ya está añadida la ciudad
lst_ciudades = [i['City']+'-'+i['Country'] for i in ciudades]

2225


In [90]:
#Si solo es un archivo de checkpoint CSV
ciudades_df = pd.read_csv('nomadlist_cities.csv').drop('Unnamed: 0', axis=1)

#generamos una lista de las ciudades para ir haciendo el append
ciudades = ciudades_df.to_dict('records')

#total de ciudades ya escrapeadas
print(len(ciudades))

#generamos una lista para verificar si ya está añadida la ciudad
lst_ciudades = [i['City']+'-'+i['Country'] for i in ciudades]

1175


In [78]:
#scroll al final de la página
n_ciudades = len(navegador.find_elements_by_tag_name('li'))

while n_ciudades<800:
    navegador.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(4)
    n_ciudades = len(navegador.find_elements_by_tag_name('li'))

In [91]:
#scraping de todas las ciudades (tarda alrededor de 10 horas)

%%time

#variable para generar checpoints
saver = 0

#loop para generar la información de cada ciudad
for i in tqdm(range(1,1350)):
    try:
        #seleccionamos una ciudad
        cada_ciudad = navegador.find_element_by_xpath(f'/html/body/div[7]/ul/li[{i}]')

        #diccionario de la ciudad
        data_ciudad = {}

        #nombre de la ciudad
        data_ciudad['City'] = cada_ciudad.find_element_by_class_name('itemName').text
        
        #nombre del pais
        try:
            data_ciudad['Country'] = cada_ciudad.find_element_by_class_name('itemSub').text
        except:
            data_ciudad['Country'] = data_ciudad['City']
        
        #si la ciudad no está en la lista (ya scrapeada), hacer el web scrapping
        if data_ciudad['City']+'-'+data_ciudad['Country'] in lst_ciudades:
            #print(data_ciudad['City']+'-'+data_ciudad['Country']+'---Ya está en lista')
            pass
        else:
            #dar click a la ciudad
            cada_ciudad.click()
            time.sleep(2)
            
            #url de la foto
            data_ciudad['Photo'] = navegador.find_element_by_xpath('/html/body/div[32]/div[3]/div[1]/img').get_attribute('src')
            
            #anexamos la ciudad y pais a la lista de ciudades scrapeadas
            lst_ciudades.append(data_ciudad['City']+'-'+data_ciudad['Country'])
            
            #####sacamos información de la ciudad

            #tab de scores
            nomad_guide = navegador.find_element_by_xpath('/html/body/div[32]/div[3]/div[2]/div[4]/ul/h3[1]')
            nomad_guide.click()

            time.sleep(1)

            tabla = navegador.find_element_by_xpath('//*[@id="body"]/div[32]/div[3]/div[2]/div[5]/div/div[1]/table/tbody')
            data = tabla.find_elements_by_tag_name('tr')

            for i in data:
                try:
                    titulo = i.find_element_by_class_name('key').text
                    titulo = re.sub('[^a-zA-Z ]',"",titulo).strip()
                except:
                    pass

                lst = ['Overall Score', 'Quality of life score', 'Family score', 'Cost',
                       'Fun', 'Safety', 'Education level', 'English speaking', 'People density /km²', 
                       'Walkability', 'Peace', 'Traffic safety', 'Hospitals', 'Happiness', 'Nightlife', 
                       'Free WiFi in city', 'Places to work from', 'A/C or heating', 
                       'Friendly to foreigners', 'Freedom of speech', 'Racial tolerance', 
                       'Female friendly', 'LGBTQ friendly', 'Startup Score']
                if titulo in lst:
                    contenido = i.find_element_by_class_name('value').text
                    if titulo == 'Overall Score':
                        contenido = contenido.split('/')[0]
                    elif titulo == 'Cost':
                        contenido = contenido.split(' ')[2]
                        contenido = re.sub('[^0-9. ]',"",contenido)
                        titulo = 'Cost/month (USD)'
                    elif i.get_attribute('data-value') != None:
                        contenido = i.get_attribute('data-value')

                    data_ciudad[titulo] = contenido

            #tab de Nomad
            nomad_guide = navegador.find_element_by_xpath('/html/body/div[32]/div[3]/div[2]/div[4]/ul/h3[2]')
            nomad_guide.click()

            time.sleep(1)

            tabla = navegador.find_element_by_xpath('//*[@id="body"]/div[32]/div[3]/div[2]/div[5]/div/div[2]/table/tbody')
            data = tabla.find_elements_by_tag_name('tr')

            for i in data:
                try:
                    titulo = i.find_element_by_class_name('key').text
                    titulo = re.sub('[^a-zA-Z ]',"",titulo).strip()
                except:
                    pass

                lst = ['Best taxi app in country','Internet speed avg','Best wireless carrier','Best wireless carrier','Tipping','Cashless society','Safe tap water','Population']
                if titulo in lst:
                    contenido = i.find_element_by_class_name('value').text
                    if titulo == 'Internet speed avg':
                        contenido = contenido.split(' ')
                        tit = contenido[1].strip()
                        titulo = f'Internet {tit}'
                        contenido = contenido[0].strip()
                    else:
                        contenido = re.sub('[^a-zA-Z0-9 ]',"",contenido).strip().split(' ')[0]
                    data_ciudad[titulo] = contenido

            #tab de Weather
            nomad_guide = navegador.find_element_by_xpath('/html/body/div[32]/div[3]/div[2]/div[4]/ul/h3[7]')
            nomad_guide.click()

            time.sleep(1)

            tabla = navegador.find_element_by_xpath('/html/body/div[32]/div[3]/div[2]/div[5]/div/div[7]/div/table/tbody')
            data = tabla.find_elements_by_tag_name('tr')

            meses = data[0].text.split(' ')
            temp = [i.split('°')[0] for i in data[2].text.split(' ')[1:] if '°' in i]

            for i,e in enumerate(meses):
                data_ciudad[f'Temperature (°C) {e}'] = temp[i]

            #agregar la información a la lista principal de ciudades
            ciudades.append(data_ciudad)

            #cerrar ciudad
            close = navegador.find_element_by_xpath('//*[@id="body"]/div[33]/div')
            close.click()
            
            #subimos la variable de checkpoints
            saver +=1
            
            time.sleep(1)
            
            #guardamos un checkpoint de csv cada 10 ciudades scrapeadas 
            if saver % 10==0:
                ciudades_df = pd.DataFrame(ciudades)
                ciudades_df.to_csv('nomadlist_cities.csv')
    except:
        pass

100%|██████████| 799/799 [1:10:33<00:00,  5.30s/it]  

CPU times: user 21.8 s, sys: 1.62 s, total: 23.4 s
Wall time: 1h 10min 33s


In [98]:
#generamos un dataframe con la lista de datos scrapeados de cada ciudad
ciudades_df = pd.DataFrame(ciudades)

In [103]:
#quitamos duplicados en caso que existan y reseteamos el indice
ciudades_df.drop_duplicates(subset=['City','Country'],inplace=True)
ciudades_df.reset_index(drop=True,inplace=True)

In [104]:
#muestra de la tabla 
ciudades_df

,City,Country,Photo,Overall Score,Quality of life score,Family score,Cost/month (USD),Internet Mbps,Fun,Safety,Education level,English speaking,Walkability,Peace,Traffic safety,Hospitals,Happiness,Nightlife,Free WiFi in city,Places to work from,Friendly to foreigners,Freedom of speech,Racial tolerance,Female friendly,LGBTQ friendly,Startup Score,Best taxi app in country,Best wireless carrier,Tipping,Cashless society,Safe tap water,Population,Temperature (°C) Jan,Temperature (°C) Feb,Temperature (°C) Mar,Temperature (°C) Apr,Temperature (°C) May,Temperature (°C) Jun,Temperature (°C) Jul,Temperature (°C) Aug,Temperature (°C) Sep,Temperature (°C) Oct,Temperature (°C) Nov,Temperature (°C) Dec
0,Lisbon,Portugal,https://nomadlist.com/assets/img/places/lisbon...,4.68,4,4,2006.0,27.0,4.0,Great,3.0,4.0,5.0,4.0,3.0,2.0,3.0,3.0,3.0,5.0,4.0,5.0,3.0,4.0,5.0,3.0,Uber,Vodafone,No,Yes,Yes,550000.0,14,16,19,18,22,21,23,25,24,21,17,16
1,"Canggu, Bali",Indonesia,https://nomadlist.com/assets/img/places/canggu...,4.61,4,4,1247.0,19.0,4.0,Great,3.0,4.0,3.0,3.0,2.0,NaN,3.0,4.0,4.0,5.0,5.0,3.0,4.0,4.0,4.0,3.0,GoJEK,XL,No,No,No,30000.0,30,30,30,30,30,28,27,27,27,29,30,31
2,Mexico City,Mexico,https://nomadlist.com/assets/img/places/mexico...,4.46,4,3,1278.0,19.0,4.0,Okay,3.0,3.0,5.0,2.0,3.0,NaN,4.0,4.0,3.0,5.0,4.0,3.0,2.0,3.0,5.0,3.0,Uber,Telcel,No,No,No,8900000.0,19,22,24,24,25,22,21,22,21,21,21,20
3,Chiang Mai,Thailand,https://nomadlist.com/assets/img/places/chiang...,4.45,4,3,1056.0,24.0,4.0,Great,3.0,3.0,5.0,3.0,2.0,5.0,4.0,3.0,4.0,5.0,4.0,3.0,2.0,4.0,3.0,3.0,Grab,AIS,No,Yes,No,400000.0,29,32,36,37,35,32,31,31,31,31,30,28
4,Belgrade,Serbia,https://nomadlist.com/assets/img/places/belgra...,4.45,4,4,1423.0,29.0,4.0,Good,3.0,4.0,5.0,3.0,3.0,NaN,3.0,5.0,4.0,5.0,4.0,4.0,3.0,4.0,1.0,3.0,CarGo,Telenor,No,Yes,Yes,1400000.0,1,8,15,18,20,28,28,32,26,23,16,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1315,"St. Cloud, MN",United States,https://nomadlist.com/assets/img/places/st-clo...,3.22,4,4,3860.0,44.0,3.0,Great,4.0,5.0,5.0,3.0,3.0,NaN,4.0,3.0,4.0,1.0,NaN,4.0,4.0,3.0,5.0,3.0,Uber,Tmobile,No,Yes,Yes,67000.0,-9,-11,0,10,15,23,26,23,20,9,1,-3
1316,Volgograd,Russia,https://nomadlist.com/assets/img/places/volgog...,3.22,3,2,1276.0,14.0,2.0,Okay,4.0,2.0,5.0,1.0,2.0,NaN,3.0,3.0,3.0,1.0,NaN,3.0,2.0,3.0,1.0,2.0,Yandex,MTS,No,Yes,No,1000000.0,-2,-2,7,15,24,30,28,26,23,17,4,2
1317,"Columbia, MO",United States,https://nomadlist.com/assets/img/places/columb...,3.22,4,3,3637.0,35.0,3.0,Good,4.0,5.0,5.0,3.0,3.0,5.0,4.0,3.0,4.0,2.0,NaN,4.0,4.0,2.0,5.0,3.0,Uber,TMobile,No,Yes,Yes,120000.0,1,1,10,18,22,27,29,27,28,17,9,8
1318,Kampala,Uganda,https://nomadlist.com/assets/img/places/kampal...,3.22,3,2,955.0,67.0,3.0,Bad,2.0,3.0,5.0,3.0,1.0,2.0,1.0,2.0,NaN,5.0,3.0,4.0,NaN,2.0,2.0,2.0,Taxify,Tecno,NaN,No,No,1500000.0,27,28,28,27,24,24,24,24,25,24,24,24


In [106]:
#guardamos el dataframe en un csv final
ciudades_df.to_csv('../data/nomadlist_cities.csv')